In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


from __future__ import print_function
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:90% !important; } </style>"))
import matplotlib.pyplot as plt
from pkg.marker_config import *
from pkg.tmp_framework import *
from pkg.constraint.constraint_action import *
from pkg.constraint.constraint_object import *
from pkg.constants import *
from pkg.utils.plot_utils import *
from pkg.utils.utils import *
from pkg.environment_builder import *
from pkg.ui.ui_broker import *
from pkg.controller.combined_robot import *
from pkg.controller.combined_robot import CombinedRobot, XYZ_RPY_ROBOTS_DEFAULT
from pkg.data_collecting.sampling import *

In [2]:
def load_dat(test_path, file_name):
    loaded_data = load_json(os.path.join(test_path, file_name))
    scene_tuple = loaded_data['scene_tuple']
    heatmap = np.array(loaded_data['heatmap'], dtype=np.float)
    heatmap_color = np.array(loaded_data['heatmap_color'], dtype=np.float)/255
    label = loaded_data['label']
    prediction = loaded_data['prediction']
    return scene_tuple, heatmap, heatmap_color, label, prediction

def get_all_cells(ghnd):
    cells = []
    for gname in ghnd.NAME_DICT.keys():
        if "cell_" in gname:
            cells.append(ghnd.NAME_DICT[gname])
    return cells

def sample_over_median(X):
    return X[np.where(X>np.median(X))]

def show_heatmap(graph, heatmap, heatmap_color, Ndwh, L_CELL, half_times=3):
    over_median = heatmap
    for _ in range(half_times-1):
        over_median = sample_over_median(over_median)
    intensity_cut = np.median(over_median)
    cells = []
    ghnd = graph.ghnd
    Nd, Nw, Nh = Ndwh
    for i in range(Nw):
        for j in range(Nd):
            for k in range(Nh):
                alpha = heatmap[i,j,k]/2
                if alpha<intensity_cut:
                    continue
                if heatmap_color is None:
                    color = (1,0,0)
                else:
                    color = tuple(heatmap_color[i,j,k])
                name = "cell_{}_{}_{}".format(i,j,k)
                pos = (np.array((i,j,k))+0.5)*L_CELL
                cell = ghnd.create_safe(name=name, link_name="base_link", gtype=GEOTYPE.BOX,
                                        center=pos, rpy=(0, 0, 0), dims=(L_CELL, L_CELL, L_CELL), color=color+(alpha,), display=True, collision=False, fixed=True)
                graph.add_marker(cell)
                cells.append(cell)
    return cells

In [3]:
MODEL_NAME = "20201224-122148/model_15"
test_path = os.path.join('logs','gradient_tape',MODEL_NAME, "test")
scene_tuple, heatmap, heatmap_color, label, prediction = load_dat(test_path, "20201212-232318/WORLD-20201212-232318/SCENE-20201212-232320/6276_heatmap_C1_0.json")

In [4]:
VISUALIZE = True
graph = None
SAMPLED_DATA = defaultdict(dict)
UPDATE_DAT = True

GLOBAL_FILENAME = "global.json"
WORLD_FILENAME = "world.json"
SCENE_FILENAME = "scene.json"
DATA_PATH = "./data"
N_retry_test = 1
LOG_DATA_PATH = "./logs/gradient_tape/"+MODEL_NAME+"/test"
TESTSET_LIST = ['20201208-121454', '20201212-232318', '20201213-061207']

In [5]:
gtimer = GlobalTimer.instance()
elog = Logger()
crob = CombinedRobot(connection_list=(False, False))
false_fail_accum = 0
false_succ_accum = 0
succ_accum = 0
fail_accum = 0

CHECK_DICT = {}

connection_list
(False, False)


In [6]:


DATASET = TESTSET_LIST[0]
CHECK_DICT[DATASET] = {}
CURRENT_PATH = os.path.join(DATA_PATH, DATASET)

## Load global params
GLOBAL_PARAMS = load_json(os.path.join(CURRENT_PATH, GLOBAL_FILENAME))
WDH = GLOBAL_PARAMS["WDH"]
L_CELL = GLOBAL_PARAMS["L_CELL"]
RATIO_DIMS = GLOBAL_PARAMS["RATIO_DIMS"]
REACH_OFFSET_DICT = GLOBAL_PARAMS["REACH_OFFSET_DICT"]
GRIPPER_REFS = GLOBAL_PARAMS["GRIPPER_REFS"]
BASE_LINK = GLOBAL_PARAMS["BASE_LINK"]
S_F_RATIO = GLOBAL_PARAMS["S_F_RATIO"]
TIMEOUT = GLOBAL_PARAMS["TIMEOUT"]

CENTER = tuple(np.divide(WDH, 2, dtype=np.float))
Ws, Ds, Hs = WDH
Nw, Nd, Nh = Nwdh = int(Ws / L_CELL), int(Ds / L_CELL), int(Hs / L_CELL)
L_MAX = L_CELL * RATIO_DIMS
print("scene size: {} ({},{},{})".format(Nw * Nd * Nh, Nw, Nd, Nh))

WORLD_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(CURRENT_PATH)))

scene size: 3375 (15,15,15)


In [7]:
WORLD = WORLD_LIST[0]
CHECK_DICT[DATASET][WORLD] = {}
WORLD_PATH = os.path.join(CURRENT_PATH, WORLD)
SAMPLED_DATA["WORLD"] = load_json(os.path.join(WORLD_PATH, WORLD_FILENAME))
Trbt_dict = SAMPLED_DATA["WORLD"]["Trbt_dict"]
reach_center_dict = {k: tuple(np.add(v[0], REACH_OFFSET_DICT[k])) for k, v in Trbt_dict.items()}

cam = None
# set urdf
xcustom, JOINT_NAMES, LINK_NAMES, urdf_content = reset_ghnd(crob.robots_on_scene, Trbt_dict,
                                                                   crob.custom_limits, start_rviz=VISUALIZE)
ghnd = GeometryHandle(urdf_content)
time.sleep(2)

# set graph
graph = TMPFramework(ghnd=ghnd, urdf_path=URDF_PATH, joint_names=JOINT_NAMES, link_names=LINK_NAMES,
                        urdf_content=urdf_content, combined_robot=crob)
graph.set_camera(cam)
graph.set_cam_robot_collision(_add_cam_poles=False, color=(1, 1, 0, 0.3))
if VISUALIZE: graph.set_rviz()

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(graph)
ui_broker.start_server()

# set rviz
if VISUALIZE: graph.set_rviz(crob.home_pose)
ui_broker.set_tables()

for gripper in GRIPPER_REFS.values():
    graph.register_binder(name=gripper['bname'], _type=FramedTool, point=gripper['tcp_ref'], rpy=[0, 0, 0],
                          link_name=gripper['link_name'])
graph.register_binder(name='base', _type=PlaceFrame, point=[0, 0, 0], rpy=[0, 0, 0], link_name=BASE_LINK)
vtem = graph.ghnd.create_safe(name="virtual", gtype=GEOTYPE.SPHERE, link_name=BASE_LINK,
                              dims=(0, 0, 0), center=(0, 0, 0), rpy=(0, 0, 0), collision=False, display=False
                              )
graph.add_object("virtual",
                 SingleHandleObject(_object=vtem,
                                    action_point=FramePoint(name="point", _object=vtem, point=(0, 0, 0),
                                                            rpy=(0, 0, 0), name_full=None)),
                 binding=("point", "base"))

obj_list = []
col_obj_list = []
SCENE_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(WORLD_PATH)))

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [8]:
SCENE = SCENE_LIST[0]
CHECK_DICT[DATASET][WORLD][SCENE] = {}
SCENE_PATH = os.path.join(WORLD_PATH, SCENE)
SAMPLED_DATA["SCENE"] = load_json(os.path.join(SCENE_PATH, SCENE_FILENAME))
Q_s = np.array(SAMPLED_DATA["SCENE"]["Q_s"])
Q_s, links, link_verts, link_ctems, link_rads = sample_joint(graph, Q_s_loaded=Q_s)
Q_s_dict = SAMPLED_DATA["SCENE"]["Q_s_dict"]
obj_dat = SAMPLED_DATA["SCENE"]["obj_dat"]

if VISUALIZE:
    graph.show_pose(Q_s)
    time.sleep(1)
    graph.show_pose(Q_s)
    time.sleep(1)
    graph.show_pose(Q_s)
for obj in obj_list: graph.remove_geometry(obj)
for odat in obj_dat:
    nbox, gtype, dims, color, center, rpy = odat["nbox"], getattr(GEOTYPE, odat["gtype"]), odat["dims"], \
                                            odat["color"], odat["center"], odat["rpy"]
    obj = graph.ghnd.create_safe(
        name="{}_{}_{}_{}".format(gtype.name, *nbox), link_name=BASE_LINK, gtype=gtype,
        center=center, rpy=rpy, dims=dims, color=color, display=True, collision=True, fixed=True)
    obj_list.append(obj)
    graph.add_marker(obj, vis=VISUALIZE)

for obj in col_obj_list: graph.remove_geometry(obj)

if VISUALIZE: graph.set_rviz()
dcol = DataCollector(graph, GRIPPER_REFS, S_F_RATIO=S_F_RATIO)
if VISUALIZE: graph.set_rviz()

# planners
mplan = MoveitPlanner(joint_names=graph.joint_names, link_names=graph.link_names, urdf_path=graph.urdf_path,
                      urdf_content=graph.urdf_content,
                      robot_names=graph.combined_robot.robot_names,
                      binder_links=[v.object.link_name for v in graph.binder_dict.values()],
                      ghnd=graph.ghnd)
dual_mplan_dict = get_dual_planner_dict(GRIPPER_REFS, graph.ghnd, graph.urdf_content, graph.urdf_path,
                                        graph.link_names, graph.combined_robot.robot_names)

In [9]:
# ## Load action
SCENE_DATA_PATH = os.path.join(LOG_DATA_PATH, DATASET, WORLD, SCENE)
DATA_LIST = filter(lambda x: x != SCENE_FILENAME, os.listdir(SCENE_DATA_PATH))
DATA_LIST = sorted(DATA_LIST, key=lambda x: int(x.split("_")[0]))
if (DATA_LIST):
    N_SET = 10
    N_dat = int(len(DATA_LIST)/N_SET)
    DATA_LIST = [DATA_LIST[i_dat*N_SET:(i_dat+1)*N_SET] for i_dat in range(N_dat)]
else:
    raise("")

In [10]:
i_dat = 10

scene_tuple, _, _, _, _ = load_dat(SCENE_DATA_PATH, DATA_LIST[i_dat][0])
ACTION = scene_tuple[4].replace(".pkl", ".json")
i_s = scene_tuple[5]
print("[BEGIN] {} - {} - {} - {} ===============".format(DATASET, WORLD, SCENE, ACTION))
snode_dict_bak = {int(k): v for k, v in load_json(os.path.join(SCENE_PATH, ACTION)).items()}
dcol.snode_dict = dcol.manager.dict()
for k, v in snode_dict_bak.items():
    dcol.snode_dict[k] = deepcopy(v)
snode_keys = sorted(snode_dict_bak.keys())

snode = dcol.snode_dict[i_s]
rname, inhand, obj, tar, dims_bak, color_bak, succ, _ = load_manipulation_from_dict(snode,
                                                                                    graph.ghnd)
if rname and tar:  # handover case
    remove_map, action_type = [[], [0, 1]], "HANDOVER"
elif inhand.collision:  # place case
    remove_map, action_type = [[], [0, 1]], "PLACE"
elif obj.collision:  # pick case
    remove_map,action_type = [[1], [0]], "PICK"
else:
    raise (RuntimeError("non-implemented case"))
print("action_type: {}".format(action_type))
print("succes: {}".format(succ))

[BEGIN] 20201208-121454 - WORLD-20201208-121454 - SCENE-20201208-121456 - 20201208-121527.json ===============
action_type: PLACE
succes: True


In [11]:
if VISUALIZE:
    show_manip_coords(graph, GRIPPER_REFS, action_type, rname, inhand, obj, rname2=tar)

In [12]:
DATA_STAGES = sorted(DATA_LIST[i_dat])

In [18]:
stage = 1
scene_tuple_, heatmap_0, heatmap_color_0, label_, prediction_ = load_dat(SCENE_DATA_PATH, DATA_STAGES[stage*2-2])
scene_tuple_, heatmap_1, heatmap_color_1, label_, prediction_ = load_dat(SCENE_DATA_PATH, DATA_STAGES[stage*2-1])
heatmap_ = np.mean([heatmap_0, heatmap_1], axis=0)
heatmap_color_ = np.mean([heatmap_color_0, heatmap_color_1], axis=0)
print("Prediction: {} ({})".format(prediction_, label_))
for cell in get_all_cells(ghnd):
    graph.remove_geometry(cell)
cells = show_heatmap(graph, heatmap_, None, heatmap_.shape, L_CELL*2**(stage-1), max(0, 4-stage))
print()

Prediction: [0.7097322344779968, -0.3902248740196228] (True)



array([[[nan]]])

In [33]:
cells = get_all_cells(ghnd)
for cell in cells:
    graph.remove_geometry(cell)

In [34]:
remove1 = [[inhand, obj][iii] for iii in remove_map[0]]
remove2 = [[inhand, obj][iii] for iii in remove_map[1]]
reset_rendering(graph, action_type, remove1, remove2, dims_bak, color_bak, sleep=True,
                vis=VISUALIZE)
graph.show_pose(Q_s)

In [90]:
x

[-3.05432619099,
 -3.05432619099,
 -3.05432619099,
 -3.05432619099,
 -3.05432619099,
 -3.75245789179,
 -2.75,
 -1.7,
 -2.75,
 -2.9,
 -2.75,
 0.1,
 -2.75,
 3.05432619099,
 3.05432619099,
 3.05432619099,
 3.05432619099,
 3.05432619099,
 3.75245789179,
 2.75,
 1.7,
 2.75,
 -0.1,
 2.75,
 3.6,
 2.75]

## testing motion planning again

In [45]:
timeout=1
if rname and tar:  # handover case
    remove_map = [[], [0, 1]]
    action_type = "HANDOVER"
    trajectory, Q_last, error, success_now = test_handover(graph, GRIPPER_REFS, rname, inhand,
                                                           obj, tar, Q_s,
                                                           dual_mplan_dict[(rname, tar)], timeout=timeout)
elif inhand.collision:  # place case
    remove_map = [[], [0, 1]]
    action_type = "PLACE"
    trajectory, Q_last, error, success_now = test_place(graph, GRIPPER_REFS, rname, inhand, obj,
                                                        tar, Q_s, mplan, timeout=timeout)
elif obj.collision:  # pick case
    remove_map = [[1], [0]]
    action_type = "PICK"
    trajectory, Q_last, error, success_now = test_pick(graph, GRIPPER_REFS, rname, inhand, obj,
                                                       tar, Q_s, mplan, timeout=timeout)
else:
    remove_map = [[], [0,1]]
    action_type = "None"
    raise (RuntimeError("non-implemented case"))
    
print(success_now)

if VISUALIZE and success_now:
    graph.show_motion(trajectory, period=0.1)

True
